In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/imdb-dataset-sentiment-analysis-in-csv-format/Valid.csv
/kaggle/input/imdb-dataset-sentiment-analysis-in-csv-format/Train.csv
/kaggle/input/imdb-dataset-sentiment-analysis-in-csv-format/Test.csv


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.metrics import accuracy_score
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Load the data

In [3]:
train=pd.read_csv('/kaggle/input/imdb-dataset-sentiment-analysis-in-csv-format/Train.csv')
test=pd.read_csv('/kaggle/input/imdb-dataset-sentiment-analysis-in-csv-format/Test.csv')
valid=pd.read_csv('/kaggle/input/imdb-dataset-sentiment-analysis-in-csv-format/Valid.csv')

In [4]:
train.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [5]:
test.head()

,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0


## Cleaning Data

In [6]:
def cleaning(text):
    text=str(text)
    text=text.lower()
    text=re.sub(r'https?://\S+|www\.\S+', '', text) ##html related tags in the string
    text = re.sub(r'[\"\#\$\%\&\'\(\)\*\+\/\:\;\<\=\>\@\[\\\]\^\_\`\{\|\}\~]',' ',text) #removing punctuation marks
    text = re.sub(r'\d+', '', text)#to remove numbers
    text = re.sub('[^a-zA-Z]+', ' ', text) #to keep only charcters
    
## remove emojis
    emojis = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',flags=re.UNICODE)
    emojis.sub(r'',text)
    
#spell correction
    text = re.sub(r'(.)\1+', r'\1\1', text)
    return text  

In [7]:
def preprocessing(text):## it text as input
    text = word_tokenize(text)##  tokenizes it into words uding word_tokenize
    text = [i for i in text if i not in stopwords]## removes any words that are found in a list of stopwords
    return ' '.join(text)##  it joins the remaining words back together into a single string.

In [8]:
train

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


## Extract text and labels

In [9]:
X_train = train['text'].values
y_train = train['label'].values
X_test = test['text'].values
y_test = test['label'].values
X_valid = valid['text'].values
y_valid = valid['label'].values

## Preprocess the text data

In [10]:
max_sequence_length = 200 
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [11]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_valid_seq = tokenizer.texts_to_sequences(X_valid)

In [12]:
X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_sequence_length, padding='post', truncating='post')
X_valid_padded = pad_sequences(X_valid_seq, maxlen=max_sequence_length, padding='post', truncating='post')

## Build the LSTM model

In [13]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train the model

In [14]:
batch_size = 64
epochs = 5

model.fit(X_train_padded, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_valid_padded, y_valid))

Epoch 1/5
625/625 [==============================] - 74s 108ms/step - loss: 0.6722 - accuracy: 0.5843 - val_loss: 0.6859 - val_accuracy: 0.5366
Epoch 2/5
625/625 [==============================] - 20s 32ms/step - loss: 0.6361 - accuracy: 0.6414 - val_loss: 0.6840 - val_accuracy: 0.5456
Epoch 3/5
625/625 [==============================] - 13s 20ms/step - loss: 0.6220 - accuracy: 0.6460 - val_loss: 0.6029 - val_accuracy: 0.7234
Epoch 4/5
625/625 [==============================] - 12s 19ms/step - loss: 0.3930 - accuracy: 0.8403 - val_loss: 0.3455 - val_accuracy: 0.8552
Epoch 5/5
625/625 [==============================] - 10s 16ms/step - loss: 0.2600 - accuracy: 0.8988 - val_loss: 0.3121 - val_accuracy: 0.8714


## Evaluate the model on test data

In [15]:
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int).flatten()
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

157/157 [==============================] - 1s 5ms/step
Test Accuracy: 87.00%


In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.87      0.87      2495
           1       0.87      0.87      0.87      2505

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000

